<a href="https://www.kaggle.com/code/vladlena321/classification-of-biology-books-log-regr?scriptVersionId=194573082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classification-of-biology-textbooks-in-russian/classification_of_biology_textbooks_in_russian.csv


In [2]:
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import joblib

import torch

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("/kaggle/input/classification-of-biology-textbooks-in-russian/classification_of_biology_textbooks_in_russian.csv", sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Название книги  1100 non-null   object 
 1   Аннотация       1100 non-null   object 
 2   Раздел          1100 non-null   object 
 3   Unnamed: 3      0 non-null      float64
 4   Unnamed: 4      0 non-null      float64
 5   Unnamed: 5      0 non-null      float64
 6   Unnamed: 6      0 non-null      float64
 7   Unnamed: 7      0 non-null      float64
 8   Unnamed: 8      0 non-null      float64
 9   Unnamed: 9      0 non-null      float64
 10  Unnamed: 10     0 non-null      float64
 11  Unnamed: 11     0 non-null      float64
 12  Unnamed: 12     0 non-null      float64
 13  Unnamed: 13     0 non-null      float64
 14  Unnamed: 14     0 non-null      float64
 15  Unnamed: 15     0 non-null      float64
 16  Unnamed: 16     0 non-null      float64
 17  Unnamed: 17     0 non-null      f

Удаляем пустые значения

In [6]:
# удаляем ненужные пустые строки

df = df.dropna(subset=['Аннотация'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1100 entries, 0 to 1099
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Название книги  1100 non-null   object 
 1   Аннотация       1100 non-null   object 
 2   Раздел          1100 non-null   object 
 3   Unnamed: 3      0 non-null      float64
 4   Unnamed: 4      0 non-null      float64
 5   Unnamed: 5      0 non-null      float64
 6   Unnamed: 6      0 non-null      float64
 7   Unnamed: 7      0 non-null      float64
 8   Unnamed: 8      0 non-null      float64
 9   Unnamed: 9      0 non-null      float64
 10  Unnamed: 10     0 non-null      float64
 11  Unnamed: 11     0 non-null      float64
 12  Unnamed: 12     0 non-null      float64
 13  Unnamed: 13     0 non-null      float64
 14  Unnamed: 14     0 non-null      float64
 15  Unnamed: 15     0 non-null      float64
 16  Unnamed: 16     0 non-null      float64
 17  Unnamed: 17     0 non-null      float6

In [7]:
# удаляем пустые столбцы

start_index = 3
end_index = 25

df = df.drop(df.columns[start_index:end_index + 1], axis=1)

df.to_csv("/kaggle/working/your_file_cleaned.csv", index=False)

In [8]:
df.head()

,Название книги,Аннотация,Раздел
0,Общая биология: основы жизни,Учебник охватывает основные концепции биологии...,Общая биология
1,Введение в биологию,Издание предлагает систематический обзор фунда...,Общая биология
2,Биология: учебное пособие,"Пособие охватывает широкий спектр тем, включая...",Общая биология
3,Основы биологии,Учебник представляет собой введение в мир живы...,Общая биология
4,Концепции современной биологии,Книга рассматривает основные принципы современ...,Общая биология


## **Модель на основе TF-IDF + Наивный Байес**

Создаем список стоп-слов, встроенной поддержки стоп-слов для русского языка в scikit-learn нет.

In [9]:
stop_words_russian = [
    'и', 'в', 'на', 'с', 'по', 'для', 'а', 'что', 'не', 'это', 'как', 'из', 'так', 'о', 'то', 'к', 'то'
]

In [10]:
vectorizer = TfidfVectorizer(stop_words=stop_words_russian)
X = vectorizer.fit_transform(df['Аннотация'])
y = df['Раздел']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.7909090909090909
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.55      0.86      0.67         7
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.91      1.00      0.95        10
                  Ботаника высших растений       0.88      0.70      0.78        10
                  Ботаника низших растений       1.00      0.80      0.89        15
Вирусология, микробиология и биотехнология       0.80      1.00      0.89         8
          Генетика и молекулярная биология       0.75      0.67      0.71         9
                   Зоология беспозвоночных       0.83      0.77      0.80        13
                      Зоология позвоночных       0.71      0.83      0.77        12
                               Иммунология       0.78      0.88      0.82         8
            История и м

## TF-IDF + Логистическая Регрессия

In [11]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8681818181818182
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.88      1.00      0.93         7
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.91      1.00      0.95        10
                  Ботаника высших растений       0.90      0.90      0.90        10
                  Ботаника низших растений       1.00      1.00      1.00        15
Вирусология, микробиология и биотехнология       1.00      1.00      1.00         8
          Генетика и молекулярная биология       0.78      0.78      0.78         9
                   Зоология беспозвоночных       0.92      0.92      0.92        13
                      Зоология позвоночных       0.82      0.75      0.78        12
                               Иммунология       0.78      0.88      0.82         8
            История и м

Кроссвалидация

In [12]:
# Настройка гиперпараметров
param_grid = {'C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшая модель после настройки гиперпараметров
best_model = grid_search.best_estimator_

# Оценка модели
y_pred = best_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Кросс-валидация
cv_scores = cross_val_score(best_model, X, y, cv=5)
print(f'Cross-Validation Accuracy: {cv_scores.mean()}')

Accuracy: 0.8681818181818182
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.78      1.00      0.88         7
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.91      1.00      0.95        10
                  Ботаника высших растений       0.90      0.90      0.90        10
                  Ботаника низших растений       1.00      1.00      1.00        15
Вирусология, микробиология и биотехнология       1.00      1.00      1.00         8
          Генетика и молекулярная биология       0.80      0.89      0.84         9
                   Зоология беспозвоночных       0.92      0.92      0.92        13
                      Зоология позвоночных       0.82      0.75      0.78        12
                               Иммунология       0.75      0.75      0.75         8
            История и м

Пайплайн

In [13]:
X = df['Аннотация']
y = df['Раздел']

# Создание пайплайна
vectorizer = TfidfVectorizer(stop_words=stop_words_russian)
model = LogisticRegression()
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', model)
])

# Обучение модели на всех данных
pipeline.fit(X, y)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words=['и', 'в', 'на', 'с', 'по', 'для',
                                             'а', 'что', 'не', 'это', 'как',
                                             'из', 'так', 'о', 'то', 'к',
                                             'то'])),
                ('classifier', LogisticRegression())])

In [14]:
joblib.dump(pipeline, 'final_model.pkl')

['final_model.pkl']

In [15]:
# Загрузка модели из файла
loaded_model = joblib.load('final_model.pkl')

# Пример использования загруженной модели
new_data = ["В монографии обобщены результаты исследований роли межполушарной асимметрии в материнско-детских взаимоотношениях млекопитающих из различных таксономических групп. Приведён обзор и критических анализ односторонних предпочтений, латерализаций, в социальном поведении в целом и в поведении матерей и детёнышей в частности. Подробно описаны методика и результаты многолетних исследований авторов по латерализации пространственных взаимоотношений матерей и детёнышей у шести видов млекопитающих: белухи, косатки, тихоокеанского моржа, сайгака, домашней лошади и серого кенгуру. Рассмотрены гипотезы о причинах возникновения односторонних предпочтений в расположении потомства относительно матери и приведены свидетельства, подтверждающие сенсорную природу данного типа латерализации. Детально рассмотрено влияние асимметричного зрительного восприятия и ведущей роли правого полушария мозга в обработке социальной информации на материнско-детские взаимоотношения. Описаны различия в поведении детёныша при разном латеральном расположении относительно матери, свидетельствующие о преимуществах латерализованного восприятия матери. Монография предназначена для специалистов, изучающих поведение, асимметрию мозга, психологию материнства, а также студентов и аспирантов биологических и медицинских факультетов."]
predictions = loaded_model.predict(new_data)

print(predictions)

['Физиология высшей нервной деятельности']


Пример использования

In [16]:
# Пример новых аннотаций для классификации
new_annotations = [
    "В учебном издании проводится краткий теоретический обзор по темам: цитологические и молекулярные основы наследственности, основам генетического анализа, хромосомной теории наследственности, генетике пола, мутационной изменчивости, биометрическим методам анализа качественных и количественных признаков, генетике популяций, биотехнологии и генетической инженерии. По всем темам и разделам пособия дан краткий теоретический обзор; задания для самостоятельной работы, задачи, контрольные вопросы. Методические указания предназначены для студентов высших учебных заведений, обучающихся по специальности 06.03.01 «Биология»."
]
# Предсказание разделов для новых аннотаций
predicted_sections = loaded_model.predict(new_annotations)

# Вывод предсказанных разделов
for annotation, section in zip(new_annotations, predicted_sections):
    print(f"Аннотация: {annotation}\nПредсказанный раздел: {section}\n")

Аннотация: В учебном издании проводится краткий теоретический обзор по темам: цитологические и молекулярные основы наследственности, основам генетического анализа, хромосомной теории наследственности, генетике пола, мутационной изменчивости, биометрическим методам анализа качественных и количественных признаков, генетике популяций, биотехнологии и генетической инженерии. По всем темам и разделам пособия дан краткий теоретический обзор; задания для самостоятельной работы, задачи, контрольные вопросы. Методические указания предназначены для студентов высших учебных заведений, обучающихся по специальности 06.03.01 «Биология».
Предсказанный раздел: Генетика и молекулярная биология

